In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator= "\n\n",
    chunk_size = 500,
    chunk_overlap = 50
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(docs, embeddings)

Created a chunk of size 682, which is longer than the specified 500
Created a chunk of size 718, which is longer than the specified 500
Created a chunk of size 614, which is longer than the specified 500
Created a chunk of size 557, which is longer than the specified 500


In [3]:
vectorstore.similarity_search("성공적인은 무엇인가?")

[Document(page_content='1. 서론: 성공적인 커리어란 무엇인가? 현대 사회에서 커리어는 단순한 생계를 위한 수단이 아니라, 개인의 삶을 구성하는 중요한 요소 중 하나다. 성공적인 커리어란 단순히 높은 연봉이나 직급을 의미하는 것이 아니라, 자신의 역량을 극대화하고 만족감을 느끼며 지속적으로 성장하는 과정을 포함한다. 그러나 많은 사람들이 커리어를 계획하는 과정에서 명확한 전략 없이 기회에 의존하거나 단기적인 목표에 집중하는 경향이 있다. 본 글에서는 성공적인 커리어를 구축하기 위한 핵심 전략을 목표 설정, 자기 계발, 네트워크 관리, 그리고 도전과 실패를 기회로 삼는 법을 중심으로 살펴본다.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='성공적인 커리어 전략: 목표 설정과 지속적인 성장의 중요성', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='3. 결론: 성공적인 커리어를 위한 실천 방법 성공적인 커리어는 단기적인 성과가 아니라, 장기적인 전략과 꾸준한 노력이 결합될 때 가능하다. 명확한 목표 설정, 지속적인 자기 계발, 의미 있는 네트워크 형성, 자기 브랜드 구축, 그리고 도전과 실패를 기회로 삼는 태도가 중요하다. 단순히 직업을 구하는 것이 아니라, 자신의 역량을 지속적으로 발전시키고 성장하는 것이 진정한 커리어 성공의 핵심이다. 결국, 중요한 것은 현재 위치가 아니라 앞으로 나아갈 방향이다. 꾸준한 노력과 전략적 사고를 바탕으로, 스스로 원하는 커리어를 개척해 나가는 것이야말로 진정한 성공이라고 할 수 있다.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='실패를 분석하라: 단순히 좌절하는 것이 아니라, 실패의 원인을 분석하고 개선할 점을 찾는 것이 중요하다. 유연한 사고를 가져라